# Исследование надёжности заёмщиков

На основе исторических данных о поведении клиентов кредитного отдела банка необходимо выяснить, как влияют и влияют ли вообще на возврат кредита следующие факторы:
- наличие детей;
- семейное положение;
- уровень дохода;
- разные цели кредита.

## Общий обзор данных

In [1]:
%pip install pymorphy2
%pip install -U pymorphy2-dicts-ru
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
import pandas as pd

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
credits_data = pd.read_csv('data.csv')

credits_data.info()
credits_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [3]:
credits_data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [4]:
credits_data[credits_data['days_employed'].isna() & credits_data['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [5]:
credits_data['children'].value_counts()
credits_data['children'] = credits_data['children'].replace(20, 2)
credits_data['children'] = credits_data['children'].replace(-1, 1)

In [6]:
credits_data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

В предоставленной таблице сразу бросается в глаза одинаковое число пропущенных значений в столбцах с данными о трудовом стаже
и доходе заемщиков. При проверке всех столбцов на неадекватные значения обнаружились отрицательные значения в столбцах `days_employed` и `children`. В столбце `days_employed` это не имеет значения, поскольку гипотезу о влиянии стажа работы на способность выплачивать кредиты мы не рассматриваем, однако исправить данные стоит. В столбце `children` также есть значение `20`, что вряд ли правда, скорее, просто опечатка. В столбце `dob_years` обнаружен 101 заемщик с возрастом 0 лет, что невозможно, в столбце `gender` есть 'XNA' — люди, не указавшие в анкете пол.

## Предобработка данных

### Обработка пропусков

Пропущенные значения — в столбцах days_employed и total_income (2174 шт.). Пропуски совпадают: если пропуск есть в days_employed, он будет и в total_income и наоборот. Причина для появления пропущенных значений может быть в том, что официальный доход заемщика слишком низок/высок/еще как-то иначе не подходит для одобрения кредита, поэтому некоторые люди предпочитают его не указывать.

Удалять строки с пропусками не станем — их около 10% от общего числа строк. Для столбца days_employed заменим отсутствующие значения средним значением по столбцу. Для total_income заменим пропуски медианами для каждого из типов занятости. Медианы выбраны как величины, на которые возможные экстремальные значения не будут влиять так сильно, как, например, на среднее.

Также есть пропуски в столбцах gender и dob_years. Строка с пропущенным значением пола всего одна. Ее отсутствие не повлияет на результаты анализа, поэтому ее можно удалить. В столбце с возрастом заменим 0 на среднее значение по столбцу.

Дополнительно исправим мусор в данных столбца days_employed: отрицательные значения заменим положительными, а имеющиеся положительные пересчитаем (судя по всему, они указаны в часах, а не в днях).

In [7]:
credits_data['total_income'] = credits_data.groupby('income_type')['total_income'].apply(lambda s: s.fillna(s.median()))
 
credits_data = credits_data[credits_data['gender'] != 'XNA']

dob_years_mean = credits_data['dob_years'].mean()
credits_data['dob_years'].replace(0, dob_years_mean)

def normalizing_function(x):
    if x < 0:
        return x * -1
    return x / 24
credits_data['days_employed'] = credits_data['days_employed'].apply(normalizing_function)

days_employed_mean = credits_data['days_employed'].mean()
credits_data['days_employed'] = credits_data['days_employed'].fillna(value = days_employed_mean)

credits_data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Стоит изменить тип данных в столбцах `days_employed` и `total_income` на «int».

### Замена типа данных

In [8]:
credits_data['total_income'] = credits_data['total_income'].astype('int')
credits_data['days_employed'] = credits_data['days_employed'].astype('int')

credits_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21524 non-null  int64 
 1   days_employed     21524 non-null  int32 
 2   dob_years         21524 non-null  int64 
 3   education         21524 non-null  object
 4   education_id      21524 non-null  int64 
 5   family_status     21524 non-null  object
 6   family_status_id  21524 non-null  int64 
 7   gender            21524 non-null  object
 8   income_type       21524 non-null  object
 9   debt              21524 non-null  int64 
 10  total_income      21524 non-null  int32 
 11  purpose           21524 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 2.0+ MB


Можно переходить к обработке дубликатов. Перед удалением дубликатов приведем к одному регистру текст в столбце `education`.

### Обработка дубликатов

In [9]:
credits_data.duplicated().sum()

54

In [10]:
credits_data['education'] = credits_data['education'].str.lower()
credits_data.duplicated().sum()

71

In [11]:
credits_data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      743
начальное                282
ученая степень             6
Name: education, dtype: int64

In [12]:
credits_data.drop_duplicates(subset=None, keep='first', inplace=True)
credits_data.duplicated().sum()

0

Изначально в таблице было 54 дубликата. Возможно, некоторые заемщики предпринимали несколько попыток взять один и тот же кредит и поэтому отправляли одну и ту же анкету, либо столкнулись с какой-нибудь технической ошибкой на сайте банка (если данные вводились через сайт), и данные были отправлены, но заемщик об этом не знал и заполнил анкету снова.

После приведения регистров к одному виду  дубликатов стало 71. Дубликаты удалены, но в столбце `purpose` имеются значения, одинаковые по смыслу, но написанные по-разному, вероятно, потому что графа «причина» — это строчка, а не выбор варианта ответа, и нет единого свода правил, что писать, если причина, например, образование: «образование», «получить образование» или «на образование». Необходима лемматизация.

### Лемматизация

In [13]:
credits_data['purpose'] = credits_data['purpose'].apply(lambda x: morph.parse(x)[0].normal_form)
                          
credits_data['purpose'].unique()

array(['покупка жильё', 'приобретение автомобиль',
       'дополнительное образование', 'сыграть свадьба',
       'операции с жилие', 'образование', 'на проведение свадьба',
       'покупка жилья для семья', 'покупка недвижимость',
       'покупка коммерческой недвижимость', 'покупка жилой недвижимость',
       'строительство собственной недвижимость', 'недвижимость',
       'строительство недвижимость', 'на покупку подержанного автомобиль',
       'на покупку своего автомобиль',
       'операции с коммерческой недвижимость',
       'строительство жилой недвижимость', 'жильё',
       'операции со своей недвижимость', 'автомобиль',
       'заняться образование', 'сделка с подержанным автомобиль',
       'получение образование', 'свадьба',
       'получение дополнительного образование', 'покупка своего жильё',
       'операции с недвижимость', 'получение высшего образование',
       'свой автомобиль', 'сделка с автомобиль', 'профильное образование',
       'высшее образование', 'покупка 

In [14]:
def new_purposes(list):
    if 'жилье' in list or 'жильё' in list or 'жилие' in list or 'жилья' in list or 'недвижимость' in list:
        return 'недвижимость'
    if 'автомобиль' in list:
        return 'автомобиль'
    if 'образование' in list:
        return 'образование'
    if 'свадьба' in list:
        return 'свадьба'
    
credits_data['purpose'] = credits_data['purpose'].apply(new_purposes)

credits_data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба


### Категоризация данных

In [15]:
debt_alert = credits_data[(credits_data['debt'] == 1)].count() / credits_data['debt'].count()
print('Процент должников среди заемщиков:', round(debt_alert['debt'] * 100, 2))

Процент должников среди заемщиков: 8.12


Данные категоризированы по одному признаку: возврат/невозврат долгов банку.
Вычисления показали, что больше 8% заемщиков не возвращают кредит в срок. Влияние разных факторов на этот показатель проверим далее.

## Анализ данных

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Разделим заемщиков в столбце `children` на 3 подгруппы: бездетные, малодетные и многодетные. Далее для каждой подгруппы узнаем процент заемщиков, имеющих задолженности по кредитам.

In [16]:
no_children = credits_data.loc[credits_data.loc[:,'children'] == 0]
print('Процент заемщиков-должников без детей:', round(no_children.groupby('children')['debt'].mean()[0] * 100, 2))

def children_amount(children):
    if 1 <= children <= 2:
        return 'малодетные'
    if 2 < children <= 5:
        return 'многодетные'
    return 'нет детей'
credits_data['children_amount'] = credits_data['children'].apply(children_amount)

few_children = credits_data.loc[credits_data.loc[:,'children_amount'] == 'малодетные']
many_children = credits_data.loc[credits_data.loc[:,'children_amount'] == 'многодетные']

print('Процент малодетных заемщиков-должников:', round(few_children.groupby('children_amount')['debt'].mean()[0] * 100, 2))
print('Процент многодетных заемщиков-должников:', round(many_children.groupby('children_amount')['debt'].mean()[0] * 100, 2))

Процент заемщиков-должников без детей: 7.54
Процент малодетных заемщиков-должников: 9.27
Процент многодетных заемщиков-должников: 8.16


Процент заемщиков-должников среди бездетных немного ниже, чем у заемщиков с детьми. При этом малодетных дожников немного меньше, чем многодетных.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [17]:
credits_data.groupby('family_status')['debt'].mean() * 100

family_status
Не женат / не замужем    9.750890
в разводе                7.112971
вдовец / вдова           6.569343
гражданский брак         9.349398
женат / замужем          7.545182
Name: debt, dtype: float64

Заемщики, не состоящие в браке и заемщики, состоящие в гражданском браке, чаще других не возвращают кредиты в срок. Разведенные заемщики и вдовы/вдовцы имеют меньше всех задолженностей по кредитам.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Чтобы понять, влияет ли уровень дохода на возврат кредита в срок, сначала выделим уровни дохода. Затем выясним процент неплательщиков в каждой из подгрупп внутри столбца `income_level`.

In [18]:
credits_data = credits_data.sort_values(by = 'total_income')
credits_data['income_level'] = pd.qcut(range(21453), 4, labels=['ниже среднего', 'средний', 'выше среднего', 'высокий'])
low_income = credits_data.loc[credits_data.loc[:,'income_level'] == 'ниже среднего']
avg_income = credits_data.loc[credits_data.loc[:,'income_level'] == 'средний']
high_avg_income = credits_data.loc[credits_data.loc[:,'income_level'] == 'выше среднего']
high_income = credits_data.loc[credits_data.loc[:,'income_level'] == 'высокий']

print('Процент должников среди заемщиков с доходом ниже среднего', low_income.groupby('income_level')['debt'].mean()[0] * 100)
print('Процент должников среди заемщиков со средним доходом', avg_income.groupby('income_level')['debt'].mean()[1] * 100)
print('Процент должников среди заемщиков с доходом выше среднего', high_avg_income.groupby('income_level')['debt'].mean()[2] * 100)
print('Процент должников среди заемщиков с высоким доходом', high_income.groupby('income_level')['debt'].mean()[3] * 100)

Процент должников среди заемщиков с доходом ниже среднего 7.960477255779269
Процент должников среди заемщиков со средним доходом 8.838336751818012
Процент должников среди заемщиков с доходом выше среднего 8.52134999067686
Процент должников среди заемщиков с высоким доходом 7.1415252657094905


Самый высокий процент должников по кредитам — среди заемщиков со средним уровнем дохода, самый низкий — у заемщиков с высоким уровнем дохода. Логично.

- Как разные цели кредита влияют на его возврат в срок?

In [19]:
credits_data.groupby('purpose')['debt'].mean() * 100

purpose
автомобиль      9.359034
недвижимость    7.234043
образование     9.220035
свадьба         8.003442
Name: debt, dtype: float64

Реже всех гасят кредиты на автомобили и образование, чаще всех — на недвижимость. Кредиты на учебу берут студенты, у них может не быть достаточного постоянного дохода — отсюда просрочки. На недвижимость кредиты взять сложнее, банки куда более тщательно проверяют доходы и кредитную историю заемщиков кредитов на несколько миллионов рублей, да и берут их, в основном, люди, твердо стоящие на ногах. Автомобили же помимо собственной стоимости требуют массу дополнительных затрат (бензин, ТО, страховка, ремонт, штрафы и многое другое), это может быть прчиной такого высокого процента невозвратов кредитов.

## Общий вывод

***Резюмируем:***

_Есть ли зависимость между наличием детей и возвратом кредита в срок?_

Есть, небольшая. Среди заемщиков с детьми в среднем на 1-2% больше должников, чем среди бездетных.

_Есть ли зависимость между семейным положением и возвратом кредита в срок?_

Есть. Неженатые/незамужние и состоящие в гражданском браке заемщики на 2% чаще не выплачивают кредит вовремя, чем женатые/замужние, и на 3% чаще, чем разведенные заемщики и вдовы/вдовцы.

_Есть ли зависимость между уровнем дохода и возвратом кредита в срок?_

Есть, небольшая. Заемщики с высоким уровнем дохода платят по кредитам лучше всех.

_Как разные цели кредита влияют на его возврат в срок?_

Лучше всех выплачиваются кредиты на недвижимость. Чуть хуже — на свадьбу. Неохотнее всего люди платят по кредитам на автомобили и образование.